# Transcribe

## 1. Package 설치

---------------------------------------

In [22]:
%load_ext autoreload
%autoreload 2

# External Dependencies:
import time
import boto3
import json
import pandas as pd
import util.xer3 as xer
import IPython.display as ipd

transcribe = boto3.client('transcribe')
s3 = boto3.resource('s3')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 2. 환경 설정

---------------------------------------

In [2]:
## 실험 1
job_name = "test-wav1"
job_uri = "s3://transcribe-dataset-cyj/contact/merge.wav"
MediaFormat = 'wav'

# 실험 2
# job_name = "test-demo1"
# job_uri = "s3://transcribe-dataset-cyj/contact/demo.mp3"
# MediaFormat = 'mp3'

# 실험 3
# job_name = "test-depart1"
# job_uri = "s3://transcribe-dataset-cyj/contact/department.mp3"
# MediaFormat = 'mp3'

## 3. Transcribe 시작

---------------------------------------

 - **MediaFileUri** : S3 내 미디어 파일 위치
 - **MediaSampleRateHertz** : 입력 파일 RateHertz 설정, 미설정 시 자동 인식
 - **MediaFormat** : 'mp3'|'mp4'|'wav'|'flac'
 - **LanguageCode** : ko-KR 언어 설정
 - **OutputBucketName** : output 버킷 설정, 미설정 시 Service managed S3 bucket에 저장 90일 내 삭제 (job 이 삭제 되기 때문임)
 - **Settings** :
    > **VocabularyName** : 고객 단어 사전 (사전 등록 필요)   
    > **VocabularyFilterName** : 삭제가 필요한 단어 (사전 등록 필요)   
    > **VocabularyFilterMethod** : remove (삭제) | mask (```***``` 로 대체)   
    > **ShowAlternatives** : 대체 단어 추천 유무 (True|False)  
    > **MaxAlternatives** : 대체 단어 추천 수 (2~ 10까지 가능)  

    > 하나만 선택 가능
    > 1. 화자 식별
    >     > **ShowSpeakerLabels** : True | False   
    >     > **MaxSpeakerLabels** : 최대 식별 화자 수 ( 2~ 10까지 가능)
    > 2. 채널 식별
    >     > **ChannelIdentification** : True | False  

<img src="./images/transcribe_할당량.png" width="600" height="400">


---------------------------------------


In [23]:
try:
    transcribe.delete_transcription_job(TranscriptionJobName=job_name)
except:
    pass

In [24]:
%%time
transcribe.start_transcription_job(
    TranscriptionJobName=job_name,
    Media={'MediaFileUri': job_uri},
#     MediaSampleRateHertz=44100,
    MediaFormat=MediaFormat,
    LanguageCode='ko-KR', ## en-US, ko-KR
    OutputBucketName='transcribe-dataset-cyj',
    Settings={
#         'VocabularyName': 'test3',
#         'ShowSpeakerLabels': True,
#         'MaxSpeakerLabels': 2,
#         'ChannelIdentification': True,
#         'ShowAlternatives': True,
#         'MaxAlternatives': 3,
#         'VocabularyFilterName': 'test-remove-voca',
#         'VocabularyFilterMethod': 'mask'
    },
#     ContentRedaction={
#         'RedactionType': 'PII',
#         'RedactionOutput': 'redacted'|'redacted_and_unredacted'
#     }
)

while True:
    status = transcribe.get_transcription_job(TranscriptionJobName=job_name)
    if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
        break
    print("Not ready yet...")
    time.sleep(5)
print(status)

Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
{'TranscriptionJob': {'TranscriptionJobName': 'test-wav1', 'TranscriptionJobStatus': 'COMPLETED', 'LanguageCode': 'ko-KR', 'MediaSampleRateHertz': 44100, 'MediaFormat': 'wav', 'Media': {'MediaFileUri': 's3://transcribe-dataset-cyj/contact/merge.wav'}, 'Transcript': {'TranscriptFileUri': 'https://s3.ap-northeast-2.amazonaws.com/transcribe-dataset-cyj/test-wav1.json'}, 'StartTime': datetime.datetime(2020, 3, 5, 0, 32, 38, 806000, tzinfo=tzlocal()), 'CreationTime': datetime.datetime(2020, 3, 5, 0, 32, 38, 774000, tzinfo=tzlocal()), 'CompletionTime': datetime.datetime(2020, 3, 5, 0, 33, 27, 202000, tzinfo=tzlocal()), 'Settings': {'ChannelIdentification': False, 'ShowAlternatives': False}}, 'ResponseMetadata': {'RequestId': '93482f31-8ce2-42a3-b6b5-60a3bf7360f8', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'appli

## 4. 결과 확인

---------------------------------------

In [5]:
response = transcribe.get_transcription_job(
    TranscriptionJobName=job_name
)
print(response)

{'TranscriptionJob': {'TranscriptionJobName': 'test-wav1', 'TranscriptionJobStatus': 'COMPLETED', 'LanguageCode': 'ko-KR', 'MediaSampleRateHertz': 44100, 'MediaFormat': 'wav', 'Media': {'MediaFileUri': 's3://transcribe-dataset-cyj/contact/merge.wav'}, 'Transcript': {'TranscriptFileUri': 'https://s3.ap-northeast-2.amazonaws.com/transcribe-dataset-cyj/test-wav1.json'}, 'StartTime': datetime.datetime(2020, 3, 4, 23, 58, 59, 708000, tzinfo=tzlocal()), 'CreationTime': datetime.datetime(2020, 3, 4, 23, 58, 59, 675000, tzinfo=tzlocal()), 'CompletionTime': datetime.datetime(2020, 3, 5, 0, 0, 39, 16000, tzinfo=tzlocal()), 'Settings': {'ChannelIdentification': False, 'ShowAlternatives': False}}, 'ResponseMetadata': {'RequestId': 'eb849af0-73a4-4252-a1fb-3fcfd605fdd4', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1', 'date': 'Thu, 05 Mar 2020 00:00:41 GMT', 'x-amzn-requestid': 'eb849af0-73a4-4252-a1fb-3fcfd605fdd4', 'content-length': '517', 'connection': 'keep-

### Output json 파일 정보

In [6]:
res = response['TranscriptionJob']['Transcript']['TranscriptFileUri']

In [7]:
tmp=res.split('/')
bucket = tmp[3]
output_filename = tmp[4]
output_path = './output/' + output_filename
print("bucket : {}, output_path : {}".format(bucket, output_path))

bucket : transcribe-dataset-cyj, output_path : ./output/test-wav1.json


### From S3 파일 to SageMaker Notebook 복사

In [8]:
s3.Object(bucket, output_filename).download_file(output_path)

In [9]:
with open(output_path, 'rt', encoding='UTF8') as f:
    print(f)
    content = json.load(f)
print(content)

<_io.TextIOWrapper name='./output/test-wav1.json' mode='rt' encoding='UTF8'>
{'jobName': 'test-wav1', 'accountId': '322537213286', 'results': {'transcripts': [{'transcript': '그녀의 사랑을 얻기 위해 애썼지만 허수 고였다. 용돈을 아껴 써라. 그는 아내를 많이 아낀 다. 그의 전화번호 나라 차에 데 잘 하세요. 거기 도착하면 난다 알려져 그들은 내가 시험에 떨어졌다고 알려왔다. 나는 살아오면서 감기를 앓은 적이 한 번도 없다. 사흘 동안 심하게 몸살을 알았어요. 요즘 공부가 안 돼요.'}], 'items': [{'start_time': '0.04', 'end_time': '0.39', 'alternatives': [{'confidence': '1.0', 'content': '그녀의'}], 'type': 'pronunciation'}, {'start_time': '0.39', 'end_time': '0.87', 'alternatives': [{'confidence': '1.0', 'content': '사랑을'}], 'type': 'pronunciation'}, {'start_time': '0.87', 'end_time': '1.26', 'alternatives': [{'confidence': '1.0', 'content': '얻기'}], 'type': 'pronunciation'}, {'start_time': '1.26', 'end_time': '1.58', 'alternatives': [{'confidence': '1.0', 'content': '위해'}], 'type': 'pronunciation'}, {'start_time': '1.59', 'end_time': '2.54', 'alternatives': [{'confidence': '0.9323', 'content': '애썼지만'}], 'type': 'pronunciat

### STT 결과

In [10]:
result = content['results']
res_item = list(result.keys())
transcript = result['transcripts'][0]['transcript']
print(transcript)

그녀의 사랑을 얻기 위해 애썼지만 허수 고였다. 용돈을 아껴 써라. 그는 아내를 많이 아낀 다. 그의 전화번호 나라 차에 데 잘 하세요. 거기 도착하면 난다 알려져 그들은 내가 시험에 떨어졌다고 알려왔다. 나는 살아오면서 감기를 앓은 적이 한 번도 없다. 사흘 동안 심하게 몸살을 알았어요. 요즘 공부가 안 돼요.


### 결과 상세 확인

In [11]:
full_items = []
items =result['items']

for item in items:
    if 'start_time' in item:
        start_time = item['start_time']
        end_time = item['end_time']
        full_items.append([start_time, end_time, item['alternatives'][0]['content'], item['alternatives'][0]['confidence']])
    else:
        full_items.append([end_time,end_time,item['alternatives'][0]['content'],item['alternatives'][0]['confidence']])

df = pd.DataFrame(full_items, columns=['start_time', 'end_time', 'content', 'confidence']) 

## 동일 결과 삭제
df = df.drop_duplicates()
df = df.drop_duplicates(['start_time','content','confidence'], keep='first')

df['start_time'] = df['start_time'].astype('float')
df['end_time'] = df['end_time'].astype('float')

df=df.sort_values(by='start_time')
df

,start_time,end_time,content,confidence
0,0.04,0.39,그녀의,1.0
1,0.39,0.87,사랑을,1.0
2,0.87,1.26,얻기,1.0
3,1.26,1.58,위해,1.0
4,1.59,2.54,애썼지만,0.9323
5,2.74,3.10,허수,0.5404
6,3.10,3.70,고였다,0.76
7,3.70,3.70,.,0.0
8,3.97,4.70,용돈을,1.0
9,4.79,5.14,아껴,1.0


### 화자 결과 확인 (optional)

In [12]:
speaker_flag = 'speaker_labels' in res_item

if speaker_flag:
    speakers = result['speaker_labels']['speakers']
    speaker_seg = content['results']['speaker_labels']['segments']
    speaker_labels = []
    for seg in speaker_seg:
        for seg_item in seg['items']:
            speaker_labels.append([seg_item['start_time'], seg_item['speaker_label'], seg_item['end_time']])

    df_speak_labels = pd.DataFrame(speaker_labels, columns=['start_time','speaker_label','end_time'])
    df_speak_labels['start_time'] = df_speak_labels['start_time'].astype('float')
    df_speak_labels['end_time'] = df_speak_labels['end_time'].astype('float')

In [13]:
if 'segments' in res_item:
    segments= result['segments']

In [14]:
if speaker_flag:
    df_result = pd.merge(df, df_speak_labels, on=['start_time', 'end_time'], how='outer')
    df_result['speaker_label'].fillna("Punc", inplace=True)
else:
    df_result= df
df_result

,start_time,end_time,content,confidence
0,0.04,0.39,그녀의,1.0
1,0.39,0.87,사랑을,1.0
2,0.87,1.26,얻기,1.0
3,1.26,1.58,위해,1.0
4,1.59,2.54,애썼지만,0.9323
5,2.74,3.10,허수,0.5404
6,3.10,3.70,고였다,0.76
7,3.70,3.70,.,0.0
8,3.97,4.70,용돈을,1.0
9,4.79,5.14,아껴,1.0


In [15]:
# out = None
# if speaker_flag:
#     out = df_result[df_result['speaker_label']=='spk_0']
# out

In [16]:
# sentence = ''
# for i in range(0, df_result.shape[0]):
#     tmp = str(df_result.iloc[i][2]) + \
#     '[' + str(df_result.iloc[i][3]) + '] '
#     if speaker_flag:
#         tmp += '[' + str(df_result.iloc[i][4]) + '] '
        
#     sentence += tmp
# print(sentence)

In [17]:
sentence = ''
pre_speaker = ''
for i in range(0, df_result.shape[0]):
    tmp = ''
    if speaker_flag:
        if df_result.iloc[i][4] not in [pre_speaker,'Punc']:
            tmp = '\n' + str(df_result.iloc[i][4]) + ' : '
        tmp += str(df_result.iloc[i][2]) +' '
        if df_result.iloc[i][4] !='Punc':
            pre_speaker = df_result.iloc[i][4]
    sentence += tmp
print(sentence)

In [18]:
print(transcript)
%store transcript

그녀의 사랑을 얻기 위해 애썼지만 허수 고였다. 용돈을 아껴 써라. 그는 아내를 많이 아낀 다. 그의 전화번호 나라 차에 데 잘 하세요. 거기 도착하면 난다 알려져 그들은 내가 시험에 떨어졌다고 알려왔다. 나는 살아오면서 감기를 앓은 적이 한 번도 없다. 사흘 동안 심하게 몸살을 알았어요. 요즘 공부가 안 돼요.
Stored 'transcript' (str)


## 5. 결과 확인
-------------------------------

In [19]:
gt ="그녀의 사랑을 얻기 위해 애썼지만 헛수고였다.용돈을 아껴 써라.그는 아내를 많이 아낀다.그 애 전화번호 알아?차에 대해 잘 아세요?거기 도착하면 나한테 알려 줘.그들은 내가 시험에 떨어졌다고 알려 왔다.나는 살아오면서 감기를 앓은 적이 한 번도 없다.사흘 동안 심하게 몸살을 앓았어요.요즘 공부가 안돼요."

In [20]:
transcript

'그녀의 사랑을 얻기 위해 애썼지만 허수 고였다. 용돈을 아껴 써라. 그는 아내를 많이 아낀 다. 그의 전화번호 나라 차에 데 잘 하세요. 거기 도착하면 난다 알려져 그들은 내가 시험에 떨어졌다고 알려왔다. 나는 살아오면서 감기를 앓은 적이 한 번도 없다. 사흘 동안 심하게 몸살을 알았어요. 요즘 공부가 안 돼요.'

In [21]:
xer.measure(transcript, gt)

CER: 16.4706%, WER: 61.5385%, SER: 100%


## 6. 결과 개선
-------------------------------

 - PCM 16비트로 녹음된 무손실 형식(예: FLAC 또는 WAV)을 사용합니다.
 - 저품질 오디오에는 8000Hz를 사용하고 고품질 오디오에는 16000Hz를 사용합니다.

2가지 방식으로 결과 개선이 가능합니다.
 - Custom vocabulary 생성  
 > - 도메인 특화 단어, 구, 단어 생성 가능  
 > - account 당 100 개 vocabulary 까지 가능  
 > - 사이즈는 최대 50KB 가능
 > - 단어 리스트로 추가하거나, 테이블 방식으로 추가 가능
 
 - Vocabulary filtering 생성  
 > - 마스크 또는 제거 방식 둘 중 선택 가능
 > - 사이즈는 최대 50KB 가능  

### Custom vocabulary 생성
--------------------------
각 단어를 한 줄씩 배치하거나 단어나 구절을 쉼표로 서로 분리하여 한 줄에 여러 단어를 배치할 수도 있습니다.
각 항목은 다음과 같이 구성되어야 합니다.
- 특정 단어 또는 구절을 대상으로 사용하기에 가장 적합
- 256자 미만(하이픈 포함)
- 허용되는 문자 집합의 문자만 가능 (한국어 가능)
- 계정당 최대 100개의 어휘 보유 가능
- 사용자 지정 어휘의 크기 한도는 50Kb
> Los-Angeles  
> F.B.I.  
> Etienne  

  > Los-Angeles, F.B.I., Etienne

In [ ]:
add_voca = transcribe.create_vocabulary(
    VocabularyName='revised-voca', ## case-sensitive
    LanguageCode='ko-KR',  ## 'ko-KR' en-US
    Phrases=[
        '엘레'
    ],
)

### Vocabulary filtering 생성

In [ ]:
remove_voca = transcribe.create_vocabulary_filter(
    VocabularyFilterName='remove-voca',
    LanguageCode='ko-KR',
    Words=[
        '스포츠',
    ],
#     VocabularyFilterFileUri='string'
)